In [1]:
import pandas as pd
from tqdm import tqdm
from IPython.display import display
import warnings
import pprint
import time
warnings.simplefilter('ignore')

#ファイルのインポート
import evaluation as ev
import control_structure as cs
import race
import table as tb
import init_table as init
import creating_test as test
import os_operation as _os_

RESULTS_FILE_NAME='buffer_2_deal_2' #.csvつけない

# インポートする dir
path='data_buffer2-loop2/'

# エクスポートする dir
results_path='results_buffer1_2/'



files=_os_.get_files(path)
send_files=files['send_files']
recv_files=files['recv_files']


heading_results=('Task','Buffer','TestCase','SVAR','Def-Use','Plnv')
heading=('Thread','Port','Event','Index')
pre_results=pd.DataFrame({},columns=heading_results)
new_results=pd.DataFrame({},columns=heading_results)

for file in tqdm(range(len(recv_files))):
    print('-------{}回目-------'.format(file+1))
    RECEIVE_SHEET_NAME=path+recv_files[file]
    SEND_SHEET_NAME=path+send_files[file]
    
    
    
    Qs=[pd.DataFrame({})]
    Qr=[pd.DataFrame({})]
    Qr[0]=pd.read_csv(RECEIVE_SHEET_NAME,names=heading)
    Qs[0]=pd.read_csv(SEND_SHEET_NAME,names=heading)
    
    init_tables=init.init_main(Qr,Qs)
    Qr=init_tables['Qr']
    Qs=init_tables['Qs']
    Qr_unique=init_tables['Qr_unique']
    Qs_unique=init_tables['Qs_unique']

    for way in range(2):
        if way==0:
            t_way=2
            combination=0 #0の場合は既存研究の方法、1の場合は組み合わせを考慮した手法
            ev_table=pre_results
        elif way==1:
            t_way=2
            combination=1
            ev_table=new_results
            Qr=Qr[:1]
            Qs=Qs[:1]
            Qr_unique=init_tables['Qr_unique']
            Qs_unique=init_tables['Qs_unique']
        else:
            print('プログラムを終了します') # shoudn't reach here
            break


        number=0 # 実験の回数
        check_digit=0

        race_set={}


        start = time.time()
        print("Creating race set",end='')
        race_set=race.creating_race_set(Qr,Qs,Qs_unique,race_set,combination)
        elapsed_time = time.time() - start
        print ("\nCreating race set took:{:.4g}".format(elapsed_time) + "[sec]")
        #pprint.pprint((race_set))


        start = time.time()
        print("Creating race table",end='')
        table=tb.construct_race_table(Qs,Qr,Qs_unique,Qr_unique,race_set,t_way)
        if t_way>1:
            table=tb.expand_table(Qr,Qs,race_set,table,t_way,Qs_unique)
        elapsed_time = time.time() - start
        print ("\nCreating race table took:{:.4g}".format(elapsed_time) + "[sec]")
        table=table.astype('int64')

        check_digit=1

        start = time.time()
        results=test.create_new_testcase(number,check_digit,Qr,Qs,table,race_set,Qr_unique,Qs_unique)
        Qr=results['recv']
        Qs=results['send']
        elapsed_time = time.time() - start
        print ("Creating test case took:{:.4g}".format(elapsed_time) + "[sec]")
        print('The number of Test Case is {}.'.format(len(Qr)))


        evaluation=ev.evaluate(Qr,Qs)
        Q=pd.concat([Qs[0],Qr[0]]).sort_index()
        display(table)
        if way==0:
            pre_results=pre_results.append(evaluation,ignore_index=True)
        elif way==1:
            new_results=new_results.append(evaluation,ignore_index=True)
        else:
            print('error')
        
_os_.save_file(RESULTS_FILE_NAME,results_path,pre_results,new_results)


100%|██████████| 5/5 [00:00<00:00, 73.49it/s]


-------1回目-------
finish initialize
Creating race set
Creating race set took:0.034[sec]
Creating race table.
Expanding table
...
Creating race table took:0.005028[sec]
Creating test case took:0[sec]
The number of Test Case is 1.


""


Creating race set
Creating race set took:0.032[sec]
Creating race table.
Expanding table
...
Creating race table took:0.007999[sec]
Creating test case took:0[sec]
The number of Test Case is 1.


""


...

finish initialize
Creating race set
Creating race set took:0.261[sec]
Creating race table....
Expanding table
......................................................................................................................................................................................
Creating race table took:10.08[sec]
.......................Creating test case took:17.34[sec]
The number of Test Case is 24.


,r1,r2,r3,r4,r5,r6,r13,r14,r15,r16,r17,r25,r26,r28
0,0,1,1,1,1,0,0,1,1,0,1,0,1,0
1,1,0,1,0,1,1,1,0,1,1,0,1,0,1
2,1,1,0,1,0,1,1,1,0,1,1,1,1,0
3,0,1,0,0,1,1,0,1,1,1,0,0,0,1
4,0,0,0,1,0,0,1,1,1,0,0,1,1,1
5,0,0,1,0,0,0,0,0,0,1,1,1,1,1
6,1,0,0,0,0,0,0,0,0,0,0,0,0,0
7,1,0,0,0,0,0,0,0,0,0,0,0,0,0
8,0,1,0,0,0,0,0,0,0,0,0,0,0,0
9,0,1,0,0,0,0,0,0,0,0,0,0,0,0


Creating race set
Creating race set took:0.376[sec]
Creating race table....
Expanding table
...............................................................................
Creating race table took:1.948[sec]
............Creating test case took:10.02[sec]
The number of Test Case is 13.


,r1,r2,r3,r4,r5,r6,r13,r15
0,0,1,1,1,1,0,0,1
1,1,0,1,0,1,1,1,0
2,1,1,0,1,0,1,1,1
3,0,1,0,0,1,1,0,1
4,0,0,0,1,0,0,1,1
5,0,0,1,0,0,0,0,0
6,1,0,0,0,0,0,0,0
7,1,0,0,0,0,0,0,0
8,0,1,0,0,0,0,0,0
9,0,1,0,0,0,0,0,0


100%|██████████| 10/10 [20:10<00:00, 121.03s/it]


In [3]:
r_last_index=len(Qr_unique)+1 #それぞれの新しいインデックスを付与するための変数→初期化
s_last_index=len(Qs_unique)+1 #+1で新しいindexをそのまま付与
for number in range(0,len(table)):
    print(".",end='')
    Qs.append(pd.DataFrame({}))
    Qr.append(pd.DataFrame({}))
    Qs[number+1]=Qs[0].copy()
    Qr[number+1]=Qr[0].copy()
    columns=list(table.columns)
    for key in range(0,len(columns)): # 列方向のループr1→r3
        if table.iloc[number][columns[key]]>0: #race_set の交換

            #Q[i+1]のテーブルを修正
            change_event=Qr[number+1][Qr[number+1]['ID']==columns[key]].iloc[0].ID # receiveの交換するやつr3
            change_event_number=Qr[number+1][Qr[number+1]['ID']==columns[key]].iloc[0].name #r3の行番号→2
            try:
                new_partner=race_set[Qr[number+1][Qr[number+1]['ID']==columns[key]].iloc[0].ID][int(table.iloc[number][columns[key]])-1] #sendの新しいパートナー s4
                new_partner_number=Qs[number+1][Qs[number+1]['ID']==new_partner].iloc[0].name #s4の行番号→3
            except:
                continue


            ## QSのindexを振りなおす処理
            new_index=[]
            for j in range(0,len(Qr[number+1])):
                if j==change_event_number:
                    new_index.append(new_partner_number)
                elif j==new_partner_number:
                    new_index.append(change_event_number)
                else:
                    new_index.append(j)

            Qs[number+1]['new_index']=new_index
            Qs[number+1]=Qs[number+1].set_index('new_index')
            Qs[number+1].sort_index(inplace=True)


            #==========Qrの重複追加作業=================

    for index,row in Qr[number+1].iterrows():
        results=cs.cstruct(Qr,Qs,Qr[number+1].iloc[index],[],check_digit,number,Qs_unique)
        judge=False
        if results: #空だったらnot 
            for index2,row2 in Qr_unique.iterrows():
                if results==Qr_unique.at[index2,'cstruct']:
                    new_index=index2
                    judge=True
                    break
            if not judge: #Falseだったら判定
                Qr[number+1].at[index,'ID']='r'+str(r_last_index)
                #Qr[number+1].at[index,'cstruct']=results
                r_last_index+=1
                temp=list(Qr[number+1].iloc[index])
                temp.append(results)
                try:
                    temp=pd.Series(temp,index=Qr_unique.columns,name=len(Qr_unique))
                except:
                    temp=pd.Series(temp,index=Qr_unique.columns[1:],name=len(Qr_unique))
                #temp=pd.DataFrame(,columns=Qr_unique.columns)
                #Qr_unique.append(temp,ignore_index=False)
                #pd.concat([Qr_unique,temp],axis=0)
                Qr_unique.loc[len(Qr_unique)]=temp
            else:
                Qr[number+1].iloc[index]=Qr_unique.iloc[new_index]
        else:
            pass

.

ValueError: Length of passed values is 7, index implies 5.

In [4]:
init_tables['Qs_unique']


['r186',
 'T3',
 18626528,
 'recv',
 5,
 'T3-5',
 ['r54', 's16', 's15', 'r2', 's2', 'r62', 's29', 's1', 's28', 'r63', 's14']]

In [7]:
init_tables['Qs_unique']

,ID,Thread,Port,Event,Index,cstruct
0,s1,T2,18626380,send,1,
1,s2,T4,18626528,send,1,
2,s3,T10,18626528,send,1,
3,s4,T6,18626380,send,1,
4,s5,T6,18626528,send,2,[s4]
5,s6,T12,18626380,send,1,
6,s7,T12,18626528,send,2,[s6]
7,s8,T8,18626380,send,1,
8,s9,T14,18626380,send,1,
9,s10,T16,18626528,send,1,


In [8]:
combination_table

NameError: name 'combination_table' is not defined

In [9]:
temp=list(Qr[number+1].iloc[index])

In [10]:
temp

['r186', 'T3', 18626528, 'recv', 5, 'T3-5']

In [11]:
temp[:5]

['r186', 'T3', 18626528, 'recv', 5]